In [1]:
%pylab notebook
import cv2
import time

Populating the interactive namespace from numpy and matplotlib


# Load the effects data

In [2]:
crown       = cv2.imread('data/crown.png', cv2.IMREAD_UNCHANGED)
glasses     = cv2.imread('data/glasses.png', cv2.IMREAD_UNCHANGED)
flower      = cv2.imread('data/flower.png', cv2.IMREAD_UNCHANGED)
eye_cartoon = cv2.imread('data/eye.png', cv2.IMREAD_UNCHANGED)
b_hat       = cv2.imread('data/b_hat2.png', cv2.IMREAD_UNCHANGED)
balloon     = cv2.imread('data/balloon.png', cv2.IMREAD_UNCHANGED)

# Load the Haar Cascades

In [3]:
face_cascade  = cv2.CascadeClassifier("utils/haarcascade_frontalface_default.xml")
eye_cascade   = cv2.CascadeClassifier("utils/haarcascade_eye_tree_eyeglasses.xml")

# Add the image categories

In [4]:
num_effects = 5
# Function   : get_option_set
# Input      : x - integer
# Output     : a set of options representing the effect
# Description: Return a set of effects to be performed
def get_option_set(x):
    return {
        0: set([]),
        1: set(['crown']),
        2: set(['flower']),
        3: set(['eye']),
        4: set(['glasses']),
        5: set(['crown', 'glasses'])
    }[x]

# Catch user input

In [5]:
# Function   : catch_ux_input
# Input      : event - Event passed by asychronous call
#              x     - x coordinate of event
#              y     - y coordinate of event
#              flags - Any special flags set
#              param - User data if any
# Output     : None
# Description: Catch the asyncrhonous user input and fill the
#              command queue based on the option entry parameter.
def catch_ux_input(event,x,y,flags,param):
    
    global option_entry
    if((x > 400) and (y > 440)):
        if event == cv2.EVENT_LBUTTONDBLCLK:
            option_entry = option_entry + 1
            option_entry = option_entry % (num_effects + 1)
            option = get_option_set(option_entry)
            cmd_q.append(option)

# Alpha blending

In [6]:
# Function   : alpha_blend
# Input      : fg - foreground with alpha channel
#              bg - background
# Output     : blended image
# Description: Blend the foreground on background
# Note       : bg and fg must have same dimensions 
def alpha_blend(fg, bg):
    b, g, r, a = cv2.split(fg)
    mask = cv2.merge((a,a,a))
    _, mask = cv2.threshold(mask,127,255,cv2.THRESH_BINARY)
    mask = mask // 255
    fg = cv2.cvtColor(fg, cv2.COLOR_BGRA2BGR)
    fg = cv2.multiply(mask, fg)
    bg = cv2.multiply(1-mask, bg)
    return cv2.add(fg, bg)

In [7]:
# Function   : set_effect
# Input      : img    - video frame
#              option - set of all effects to be performed
# Output     : output frame
# Description: Perform the requested effect on the video frame
def set_effect(img, option):
    
    # Compute the gray scale version of frame
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Find all the faces in the frame
    faces = face_cascade.detectMultiScale(gray, 1.3, 3)
    
    # For each face, apply the associated effects (if any)
    for (x,y, w, h) in faces:
        roi_g = gray[y:y+h, x:x+w]
        roi_c = img[y:y+h, x:x+w]
        
        # Fetch the effect image as foreground
        if('crown' in option or 'flower' in option or 'birthday' in option):
            if('crown' in option):
                fg = crown.copy()
            elif('flower' in option):
                fg = flower.copy()
            elif('birthday' in option):
                fg = b_hat.copy()
                
            # Scale the foreground wrt face size
            sf = w / fg.shape[1]
            dim = (int(round(fg.shape[1]*sf)), int(round(fg.shape[0]*sf)))
            fg_rz = cv2.resize(fg, dim)
            
            # Extract the background frame to perform alpha blending
            back = img[y-fg_rz.shape[0]:y, x:x+fg_rz.shape[1]]
            img[y-fg_rz.shape[0]:y, x:x+fg_rz.shape[1]] = alpha_blend(fg_rz, back)

        
        # Fetch the combination of both eye effect images as foreground
        if('glasses' in option or 'eye' in option):
            if('glasses' in option):
                fg = glasses.copy()
            elif('eye' in option):
                fg = eye_cartoon.copy()

            # Find the eyes in the face region of interest
            eye = eye_cascade.detectMultiScale(roi_g, 1.1, 3)
            
            # To avoid multiple false positives
            if(len(eye) == 2):
                
                # Find the width in accordance to distance between the eye's end to end
                w = abs(eye[0][0] - eye[1][0]) + max(eye[0][2], eye[1][2])
                x = min(eye[0][0], eye[1][0])
                y = max(eye[0][1], eye[1][1])
                
                # Scale the foreground according to requirement
                sf = w / fg.shape[1]
                dim = (int(round(fg.shape[1]*sf)), int(round(fg.shape[0]*sf)))
                n_dim = (int(round(dim[0]*1.2)), int(round(dim[1]*1.2)))
                hdiff_x = int(round((n_dim[0] - dim[0])/1.8))
                hdiff_y = int(round((n_dim[1] - dim[1])/1.8))
                
                # Adjust some delta to accomodate more natural looking effects
                y_pl = y - hdiff_y
                x_pl = x - hdiff_x
    
                fg_rz = cv2.resize(fg, n_dim)
        
                # Get the background roi to perform alpha blending
                back = roi_c[y_pl: y_pl+n_dim[1], x_pl:x_pl+n_dim[0]]
                roi_c[y_pl: y_pl+n_dim[1], x_pl:x_pl+n_dim[0]] = alpha_blend(fg_rz, back)

    return img            

# Start the video capture


In [8]:
# Use webcam or input file based on requirement
filename = input("Press Enter to use camera or input file name")
if(filename == ''):
    filename=0
    print("As per user input: using webcam")

# Open the video capture stream
test = cv2.VideoCapture(filename)

# Set the option_entry and command queue as global to be used for mouse callback
global option_entry, cmd_q
option_entry = 0
cmd_q = []

# Set the mouse callback
cv2.namedWindow('frame')
cv2.setMouseCallback('frame',catch_ux_input)

# For calculating FPS
prev_time = time.time()
num_frames = 0

# Dummy option set
option = set([])

Press Enter to use camera or input file name
As per user input: using webcam


In [9]:
while(True):
    
    num_frames = num_frames + 1
    
    # Read the input stream frame
    ret, frame = test.read()
    
    # If there is command effect pending, do that
    if(len(cmd_q) > 0):
        option = cmd_q.pop(0)
    
    try:
        
        # Set the effect
        out = set_effect(frame, option)
        
        # Print the option for user to provide input
        font = cv2.FONT_HERSHEY_SCRIPT_COMPLEX
        cv2.putText(out,'Change Effect',(400,465), font, 1.2,(127,127,255),2,cv2.LINE_AA)
        
        # Resize the OpenCV window for better visualization
        out = cv2.resize(out, (1200, 900))
        cv2.resizeWindow('frame', 1200,900)
        
        # Display the live effects to the user
        cv2.imshow('frame',out)
    except:
        pass
    
    # Continue looping till user presses 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Compute the frame rate.
tic = time.time()
print("FPS: ", num_frames/(tic - prev_time))

# Release the capture object and destroy all open OpenCV windows
test.release()
cv2.destroyAllWindows()

FPS:  13.74292868764855
